<p align="center">
  <img src="https://huggingface.co/speakleash/Bielik-7B-Instruct-v0.1/raw/main/speakleash_cyfronet.png">
</p>

### Przygotowanie środowiska

W celu instalacji niezbędnych bibliotek z ograniczoną ilością zwracanych informacji, można:
- zastosować flagę '-q' (quiet) dla przy każdej bibliotece, która sprawi, że wyświetli się mniej informacji
- użyć komendy %%capture na początku komórki, która przechwytuje wyniki wykonywania kodu w danej komórce

In [20]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [21]:
import torch

In [22]:
import warnings
warnings.filterwarnings("ignore") # tylko na potrzeby naszego demo ;)

### Inicjalizacja modelu (4 bit)

In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = 'speakleash/Bielik-7B-Instruct-v0.1'
load_4_bit = True

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

quantization_config = None

if load_4_bit:
    quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16
    )

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.bfloat16,
                                             quantization_config=quantization_config
                                             )

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Funkcja pomocnicza do generowania odpowiedzi

In [24]:
import uuid
import json

In [25]:
temperature = 0.7
max_tokens = 256
top_k = 100
top_p = 1


def generate(prompt, system=None):
    messages = []

    if system:
        messages.append({"role": "system", "content": system})

    messages.append({"role": "user", "content": prompt})

    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = input_ids

    # Sprawdzenie dostępności CUDA i przeniesienie danych wejściowych na odpowiednie urządzenie (GPU lub CPU)
    if torch.cuda.is_available():
        model_inputs = input_ids.to(device)

    # generowanie odpowiedzi
    outputs = model.generate(model_inputs,
                             streamer=streamer,
                             max_new_tokens=max_tokens,
                             do_sample=True if temperature else False,
                             temperature=temperature,
                             top_k=top_k,
                             top_p=top_p)

    # zapis do pliku
    filename = f"output{str(uuid.uuid4())[:8]}.txt"

    with open(filename, "w") as file:
        content = {
                "prompt": messages,
                "output": tokenizer.batch_decode(outputs, skip_special_tokens=False)
        }
        json.dump(content, file, ensure_ascii=False, indent=4)

#### Test

In [26]:
generate("Hej, jesteś tam?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Cześć! Tak, jestem tutaj. W czym mogę Ci pomóc?


In [27]:
generate("Hej, czym się zajmujesz?", system = "Jesteś programistą baz danych, który chce rozwijać swoje kompetencje w obszarze AI.")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Jako programista baz danych specjalizuję się w projektowaniu, tworzeniu i zarządzaniu bazami danych. Współpracuję z klientami w celu identyfikacji ich potrzeb, definiowania struktury danych, implementowania systemów bazodanowych i zarządzania nimi. Moja praca może obejmować różne obszary, od analizy danych po optymalizację wydajności i bezpieczeństwa baz danych. Zazwyczaj jestem konsultantem ds. baz danych, a czasem pełnię rolę lidera projektu. 

Ostatnio moje zainteresowania poszerzyły się o sztuczną inteligencję (AI). Widzę potencjał w wykorzystywaniu AI do ulepszania tradycyjnych systemów bazodanowych. Chcę rozwijać swoje


# Przykład 1

Po prostu **"Hello World"**, a bardziej **"Witaj świecie!"**

In [28]:
temperature = 1.0
max_tokens = 256

prompt = "Witaj świecie!"
generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Witaj! Cieszymy się, że do nas dołączyłeś. Jeśli masz jakieś pytania lub potrzebujesz pomocy, służymy pomocą!


# Przykład 2

Zadawanie pytań otwartych

In [29]:
temperature = 0.1
max_tokens = 512

prompt = "Co to jest fotosynteza?"

In [30]:
generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Fotosynteza to proces, w którym rośliny, glony i niektóre bakterie przekształcają energię słoneczną w chemiczną energię w postaci glukozy, wykorzystując do tego chlorofil - zielony barwnik obecny w chloroplastach. W trakcie tego procesu rośliny pobierają wodę i dwutlenek węgla z otoczenia, a następnie wykorzystują energię słoneczną do przekształcenia tych składników w glukozę i tlen. Glukoza jest źródłem energii dla roślin, a tlen jest uwalniany do atmosfery jako produkt uboczny. Fotosynteza jest kluczowa dla życia na Ziemi, ponieważ dostarcza tlenu do atmosfery i jest podstawą łańcucha pokarmowego.


# Przykład 3

Praca na tekście

In [31]:
temperature = 0.1
max_tokens = 512

In [32]:
review = "Przy waszych odsetkach to tylko chleb i margaryna!"
prompt = f"Czy podana opinia jest pozytywna czy negatywna i wyjaśnij to obszernie.\n\n###OPINIA:\n{review}"

generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Opinia jest negatywna. Autor użył sformułowania "odsetkach" w sposób ironiczny, sugerując, że poziom intelektualny osoby, do której się odnosi, jest niski lub ograniczony. Ponadto, użycie słowa "chleb" w tym kontekście może wskazywać na brak wyrafinowania lub zrozumienia. Użycie tych słów razem sugeruje, że autor uważa tę osobę za mało inteligentną lub ograniczoną intelektualnie. Jest to negatywna ocena, wyrażona w sposób pośredni poprzez ironię i użycie pewnych słów w sposób krytyczny.


In [33]:
system = "Pracujesz jako specjalista od zarządzania jakością w banku i badasz doświadczenia klientów. Analizujesz opinie, wskazujesz pozytywne aspekty, na które zwrócili uwagę klienci oraz obszary do poprawy. Odpowiadasz w języku polskim."

generate(review, system)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Dziękujemy za wyrażenie swojej opinii. Jesteśmy świadomi, że może być obszar do poprawy w obszarze odsetek. Cieszymy się, że zwróciłeś uwagę na te aspekty i doceniamy Twoje zaangażowanie w doskonalenie naszych usług. Będziemy kontynuować monitorowanie Twoich doświadczeń oraz wszystkich naszych klientów, aby stale ulepszać jakość naszych produktów i usług.

Jeśli masz jakieś konkretne sugestie dotyczące tego, jak możemy poprawić doświadczenia naszych klientów, prosimy o kontakt. Chcemy zapewnić, że Twoje zdanie jest dla nas ważne i pomoże nam dostosować się do potrzeb naszych klientów.


In [34]:
temperature = 0.2

review = """Aplikacja działała świetnie i jest już w języku angielskim...​
Jednak od tygodnia zawiesza się, nie można się zalogować, używam tylko blika i akceptuję płatności...​
Kiedy to zostanie rozwiązane?"""

prompt = f"Czy podana opinia jest pozytywna czy negatywna? Uzasadnij odpowiedź, wymień w punktach aspekty pozytywne oraz wymagające poprawy. \n\n###OPINIA:\n{review}"

generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Opinia jest negatywna z elementami pozytywnymi.

Aspekty negatywne:

1. Aplikacja działa źle - od tygodnia się zawiesza.
2. Utrudniony dostęp - nie można się zalogować.
3. Ograniczone metody płatności - tylko BLIK jest akceptowany.
4. Brak informacji o planowanym terminie naprawy.

Aspekty pozytywne:

1. Aplikacja działa w języku angielskim, co jest pozytywnym akcentem, jednak problematyczne jest, że obecnie się zawiesza.
2. Dostępność metody płatności BLIK.


# Przykład 4
Praca z dokumentami

In [35]:
import requests
from bs4 import BeautifulSoup


def get_page_content(url='https://hiperautomatyzacja.pl'):
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    plain_text = soup.get_text()
    clean_text = plain_text
    return clean_text

In [36]:
temperature = 0.1
max_tokens = 512

system = "Dokonaj streszczenia podanego tekstu strony internetowej. Pisz w języku polskim."
prompt = get_page_content("https://hiperautomatyzacja.pl")

generate(prompt, system=system)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 HiperAutomatyzacja.pl to strona internetowa poświęcona konferencji i społeczności hiperautomatyzacji i AI w Polsce. Organizator konferencji oferuje nieodpłatny udział w konferencji, podczas której prelegenci będą dzielić się swoimi praktycznymi doświadczeniami. Konferencja podzielona jest na dwa strumienie - menadżerski i wdrożeniowy. Uczestnicy otrzymują dostęp do nagrań z konferencji, a także biorą udział w radarze ułatwiającym wdrażanie AI w organizacjach. Konferencja jest częścią większego projektu budowy polskiej społeczności hiperautomatyzacji i AI - Liderzy.AI.


In [37]:
prompt = get_page_content("https://hiperautomatyzacja.pl/program?s=7")
generate(prompt, system=system)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 HiperAutomatyzacja.pl to platforma poświęcona automatyzacji procesów biznesowych. Na stronie znajduje się program o nazwie HiperAutomatyzacja, który koncentruje się na pomocy firmom w optymalizacji pracy poprzez automatyzację. Oferowane są transmisje, szkolenia, a także narzędzia i platformy do wdrażania rozwiązań RAG. Strona porusza również tematy takie jak sztuczna inteligencja, RPA i AI w kontekście optymalizacji procesów. Dostępne są transmisje i nagrania wideo, a także publikowane są artykuły i raporty dotyczące automatyzacji. Organizator strony to grupa ekspertów, którzy dzielą się swoją wiedzą i doświadczeniem z firmami chcącymi zoptymalizować swoje procesy.


In [38]:
system = "Wymień warsztaty dotyczące AI oraz prowadzących z imienia, nazwiska z poniższego tekstu strony internetowej."
prompt = system + "\n\nTEKST:" + get_page_content("https://hiperautomatyzacja.pl/program?s=7")
generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Poniżej przedstawiona jest lista warsztatów AI prowadzonych przez wymienione w tekście osoby:

1. "Moc Inteligentnych Procesów" - prowadzący: Zbigniew Kępiński (Grupa Raben, Organizator)
2. "Hiperautomatyzacja w logistyce (10/5/1)" - prowadzący: Zbigniew Kępiński (Grupa Raben, Organizator)
3. "Transmisja" - prowadzący: Zbigniew Kępiński (Grupa Raben, Organizator)
4. "Prototypowanie automatyzacji procesów z wykorzystaniem GenAI" - prowadzący: Tomasz Gawron (XTB, Organizator)
5. "Platformy Automatyzujące w skali Enterprise" - prowadzący: Maciej Zyzak (ING Hubs Poland, Organizator)
6. "Symbioza gatunków - RPA i AI w PZU" - prowadzący: Maciej Rudziński (PZU, Organizator)
7. "Podejście do wdrożenia governance nad dużymi modelami językowymi i sztuczną inteligencją w Power Platform" - prowadzący: Tomasz Łopuchowicz (ISS World Services, Organizator)
8. "Czy agenci AI zastąpią RPA? - Przegląd technologii i narzędzi AI do automatyzacji" - prowadzący: Krzysztof Karaszewski (Międzynarodowy Bank, 

In [39]:
system = """Wymień warsztaty dotyczące AI z poniższego tekstu strony internetowej. Zwróć w postaci JSON według szablonu
[
  {
    'trener':...,
    'temat':...,
    'godzina': ...
    }
    ,
] """

prompt = system + "\n\nTEKST:" + get_page_content("https://hiperautomatyzacja.pl/program?s=7")
generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 {
 "trener": {
   "name": "Zbigniew Kępiński",
   "event": "Moc Inteligentnych Procesów",
   "date": "10/5/1",
   "time": "09:10-10:10"
 },
 "trener": {
   "name": "Tomasz Gawron",
   "event": "Transmisja",
   "date": "10/5/1",
   "time": "10:50-11:30"
 },
 "trener": {
   "name": "Maciej Zyzak",
   "event": "Platformy Automatyzujące w skali Enterprise",
   "date": "10/5/1",
   "time": "11:50-12:30"
 },
 "trener": {
   "name": "Maciej Rudziński",
   "event": "Podejście do wdrożenia governance nad dużymi modelami językowymi i sztuczną inteligencją w Power Platform",
   "date": "10/5/1",
   "time": "12:50-13:30"
 },
 "trener": {
   "name": "Artur Szylkin",
   "event": "Symbioza gatunków - RPA i AI w PZU",
   "date": "10/5/1",
   "time": "13:30-14:10"
 },
 "trener": {
   "name": "Krzysztof Karaszewski",
   "event": "Czy agenci AI zastąpią RPA? - Przegląd technologii i narzędzi AI do automatyzacji",
   "date": "10/5/1",
   "time": "14:10-14:50"
 },
 "trener": {
   "name": "Paweł Bulowski",

In [40]:
max_tokens = 1024
temperature = 0.1

system = "Pracujesz w agencji rekrutacyjnej. Pomagasz dużej firmie z branży finansowej zatrudnić menedżera do spraw AI. Dokonaj oceny podanego kandydata i wypisz jego mocne strony."
prompt = get_page_content("https://hiperautomatyzacja.pl/o-organizatorze")
generate(prompt, system=system)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Mocne strony kandydata:

1. Głęboka wiedza i doświadczenie w zarządzaniu projektami: Kandydat posiada ponad 15 lat doświadczenia w zarządzaniu projektami w różnych branżach, co pozwala mu na efektywne planowanie, koordynację i kontrolę projektów.

2. Silne umiejętności analityczne i strategiczne: Kandydat posiada wysoko rozwinięte umiejętności analizy biznesowej i strategicznego myślenia, co pozwala mu na identyfikowanie możliwości i ryzyk oraz tworzenie efektywnych planów działania.

3. Wszechstronność technologiczna: Kandydat jest biegły w różnych technologiach, w tym w zarządzaniu projektami, analizie danych, zarządzaniu zespołami zdalnymi oraz tworzeniu i wdrażaniu strategii cyfryzacji.

4. Umiejętność budowania relacji i zespołów: Kandydat posiada wysoko rozwinięte umiejętności interpersonalne, co pozwala mu na skuteczne budowanie relacji z klientami, partnerami biznesowymi oraz zespołami projektowymi.

5. Silne umiejętności komunikacyjne i prezentacyjne: Kandydat posiada wysoko 